* 关系型数据库

注意ACID

* 关系操作
  - 选择 (selection): 又称为限制，是从关系中选择除满足给定条件的元组
  - 投影 (projection)： 
  - 连接 (join)
  
* 关系  
  * 员工表id、部门表id
  * 主键: 从候选键中选择任意一个就可以称为主键。主键约束 唯一且不为null（不能同名 不能为空） 
  * 候选键：1个字段或者多个字段联合在一起，可以唯一的确定一条记录
  
多对多： 1个 员工 可以属于多个 部门， 1个 部门 可以有n个 员工。 使用员工ID和部门ID构建第三张表
  * 解决：引入第三张表 设置员工ID 和 部门ID的对应关系

一对多： 1个 员工 可以属于1个 部门， 1个 部门 可以用n个 员工。 
  * 解决: 在员工表 建立**外键** 部门_id，要引用主表部门表的主键id 

一对一： 1个 部门 只能有1个 领导， 1个 领导 只能管1个 部门
  * 解决：建立外键 从表中建立

primary key: id
name: unique key  # 唯一键约束 可以为空，但不能重复. null设置False -> name称为候选键

* 实践
  - 主键：建表的时候 包括Dango 推荐二话不说上来就一个id, 自增主键
    - 单一字段主键（ID）, 方便
    - 联合主键：多行一起才可以唯一确定一行
  - 外键：都是引用其他表 主键(按道理 候选键也行 推荐使用主键 方便)
  - 更新：update 一定要指定条件 否则会全表更新 **危险**
  - 删除：delete 一定要指定条件 否则会全表删除 **危险** 建议使用假删除 设置字段 isDeleted 0 -> 1 删除数据：isDeleted 置为1

```sql
update register set isDeleted = 1 where id = 100;
```

dept_emp
id    dept_id(F)    emp_id(F)
100      1            1
5        2            1
10       2            2

id：单一字段主键，方便
dept_id emp_id: 候选键，外键引用(数据来源不属于自己，参照别人主键)和约束(参照的表主键有 才可以)



* 范式
  - 目的 -- 减少冗余，导致数据库内容精简，从而占用更小的存储空间
  - 第1、2、3范式，一般建议达到第三范式
  - 一般来说，尽量达到第三范式是好的习惯
  - 但是，在注重**性能**的时候，可以用空间换时间，增加冗余来换取更高的性能
    - 将常用的各个表数据放到一张表 用冗余数据来减少我们对多表联合查询时的性能损耗
    - 同一个数据 在多张表中冗余 问题：更新一处，为了保持一致 就必须更新所有处（一处更新 处处更新）

* DML 数据库操纵语句的学习 （重点）
  - **select**
  - update
  - delete
  - insert
  
增删改，查询 性能不能兼得
增删改性能低，查询性能高

大多数请况下遇到的库：读多写少   -- 做查询性能提高(比如：cache)

In [ ]:
* 单表查询

* SELECT
井号在sql中也表示注释 但是标准时使用： -- sql

```sql
SELECT * FROM employees; -- 生产环境中不要使用这句话 因为没有条件 没有指定字段

-- projection 投影：选择特定字段 而不是*
SELECT emp_no, first_name, last_name FROM employees; 

-- + 加号用在数值相加
-- CONCAT函数 连接字符串；sql内建函数
-- 别名：name 后面的编程工具使用的 as关键字可以省略（建议加上）
-- select 投影的字段 可以使用as
SELECT emp_no as id, CONCAT(first_name, ' ', last_name) as name FROM employees;

-- from 后面的表名 可以使用as
-- 从单表获取数据的时候 表名可以省略；多张表 不能省略；一旦写了别名 必须使用别名 否则会报错 找不到表
-- 生产环境 不推荐这么使用 不要看全表数据(数据量非常大的时候 灾难性后果)
SELECT emps.emp_no as id, CONCAT(emps.first_name, ' ', emps.last_name) as name from employees as emps;


-- pagination 分页查询 必须的：分页时查询的常态  limit&offset
SELECT emps.emp_no as id, CONCAT(emps.first_name, ' ', emps.last_name) as name from employees as emps
-- LIMIT 5    -- 返回多少个
-- OFFSET 20  -- 偏移多少
limit 2, 5   -- 第二种写法：偏移1个 取5个


-- seltction 选择 按照给的条件过滤结果
-- 不等于 mysql标准写法：<>  也支持：!=
-- 使用 emp_no > 10006 没有用 id > 10006(使用id报错) 执行的子句是要分先后的
-- 子句是有先后的 from --> where --> projection
-- 1. 先加载表 2. 选择 3. 挑字段 4. 投影 比较靠后 
-- emp_no是主键 MySQL是按照主键组织存储数据的 B+tree
SELECT emp_no as id, CONCAT(first_name,' ',last_name) as name FROM employees
-- WHERE emp_no > 10006 AND emp_no < 10009  -- (10006, 10009)
-- WHERE emp_no BETWEEN 10006 AND 10009 OR emp_no = 10001 -- [10006, 10009]
-- not in 要小心 反选大多数时候 范围更大 相当于很多条数据库选择 增加数据库压力
-- select 要注意筛选范围
-- 最优使用主键筛选, 使用带键字段筛选(主键>唯一键) 使用索引，其他一般不推荐 相当于全表扫描 很慢
-- WHERE emp_no in (10001, 10006, 10007, 201389138)
WHERE last_name LIKE BINARY 'P___' -- p字符开头 左前缀匹配：姑且用一下 其他的性能低
-- 字符串匹配 能不做尽量不做
-- 解决的技术：加索引 不要在mysql中用 在ES中用：ELK中的ES 建倒排索引，海量数据数据库中的字符串全文搜索、模糊匹配(分词 按词段的)  
-- 并发小：无所谓 并发大：上索引 ES倒排索引(solr)  
-- % mysql通配符 任意字符  LIKE: 忽略大小写 LIKE BINARY: 区分大小写
-- _ 代表一个字符

```

* order by 按照某一字段排序

```sql
SELECT emp_no as id, CONCAT(first_name,' ',last_name) as name FROM employees
WHERE emp_no > 10015
ORDER BY emp_no DESC 
-- 默认升序(ASC) DESC: 降序 
-- order by的字段 未必投影(排序靠后 可以使用 select as 的别名)
-- 可以利用投影的结果、别名

SELECT emp_no, dept_no from dept_emp
WHERE emp_no < 10012 and emp_no > 10008
ORDER BY emp_no DESC, dept_no  -- 第一字段相等时 才按照第二字段来排序
```

* distinct 去重：不返回重复记录
```sql
SELECT DISTINCT emp_no, dept_no from dept_emp  -- DISTINT 去重 按照出来的数据 判断是否需要去重
WHERE emp_no < 10012 and emp_no > 10008
```

* 聚合函数

```sql
SELECT count(first_name), max(emp_no), min(emp_no), sum(emp_no), AVG(emp_no) FROM employees;  
-- 常用聚合函数 employees表中 总记录 最大|最小工号 求和 求平均
-- 统计出来几行 1行;count可以使用 * 或者 主键 但是一定不要使用其他键
-- count 配合分页使用
```

* 分组 group by

```sql
SELECT avg(emp_no), gender, count(*) from employees
WHERE emp_no < 10006
GROUP BY gender;  -- 分组之后再统计 GROUP BY 在投影前面
-- GROUP BY 按照该记录字段(分类) 相同的分为一类
-- 非分组字段显示是没有实际含义的
-- 塌缩 遮蔽 非分组字段选第一个代表显示
-- 分组几个 结果出几行(分组时可以使用聚合函数)

SELECT emp_no, count(*), avg(salary) as avg_sal FROM salaries
WHERE emp_no < 10003
GROUP BY emp_no
HAVING avg_sal > 45000  -- 不统计不知道 只能在HAVING做 
-- 条件 可以在WHERE做(对原始数据做处理) 也可以在 HAVING中做(对统计结果做处理)
-- 能在where做 就在where做 先过滤最好(保证数据越来越少 提高性能)
ORDER BY avg_sal
LIMIT 1  # top N

SELECT count(*), max(emp_no), min(emp_no), sum(emp_no), avg(emp_no)
FROM employees;
-- 使用了内建函数 就是使用了分组 只是没给条件而已(没给条件 默认对全表分组)

SELECT emp_no, min(salary) FROM salaries;
-- 内建函数就使用了分组 此时 使用emp_no(非分组字段)没有意义

SELECT emp_no, min(salary) as m FROM salaries
GROUP BY emp_no
ORDER BY m
LIMIT 1  -- 拿最小工资 根据emp_no分组 得到N组 再排序取最小的sal和emp_no
```

* select语句 使用别名
from > where > projection   # 投影

先from加载表 -- 再按照where条件子句选择行 -- 再做投影选择字段

In [ ]:
* 子查询

* 常用

IN子查询 用的比较多
子查询不要使用order by 没有意义

```sql

-- 有工资的员工是谁
SELECT * FROM employees
WHERE emp_no IN (SELECT DISTINCT emp_no FROM salaries);  -- emp_no 最好去重DISTINCT
```

* 不常用

```sql
SELECT emps.emp_no, emps.`name`, emps.gender FROM
(SELECT emp_no, CONCAT(first_name,' ',last_name) as name, gender 
FROM employees 
WHERE emp_no > 10017) as emps;
```

In [ ]:
* JOIN
* 重点 必用

* join 连接

```bash
cross inner 可以省略不写
不写默认为：cross连接 
加on条件为：inner连接 

on 关键字 跟着join走 不能随便使用
```

```sql
SELECT * FROM employees;  -- 20 record
SELECT * FROM salaries; -- 40 recor
```

  - cross JOIN   
    - 交叉连接，迪卡尔乘积(两个集合里面的所有record 相乘) 基本不用 非常耗资源 
    - 其他连接 一点要**加条件** on语句加条件 否则后果非常恐怖
  ```sql
  SELECT * FROM employees CROSS JOIN salaries;  -- 20x40=800 record
  A  B
  a  1  a1 a2 a3
  b  2  b1 b2 b3
         3
   ```

  - INNER JOIN   
    - 内连接 必须加on 等值连接, 用的比较多
  ```sql
      -- 内连接不加条件 就等于交叉连接
      SELECT * FROM employees as e INNER JOIN salaries as s
on e.emp_no = s.emp_no;  -- 两张表 连接emp_no相等的记录
      
      SELECT * FROM employees as e INNER JOIN salaries as s
on e.emp_no = s.emp_no and e.emp_no = 10002;

      SELECT * FROM employees as e INNER JOIN salaries as s
on e.emp_no = s.emp_no
WHERE e.emp_no = 10002;  -- 和上面的等值条件判断效果一样 性能也差不多

      SELECT * FROM employees e, salaries s  -- 省略 join使用逗号 默认交叉join 不能使用 on关键字
WHERE e.emp_no = s.emp_no and e.emp_no = 10002;

  ```
  
  - NATURL JOIN  
    - 自然连接 会去掉多余的行 很自然的连接 不常用 复杂的等值关系无法表达
  ```sql
  SELECT * FROM employees e NATURAL JOIN salaries s;
  ```
  
  - outer JOIN   外连接（左外连接 右外连接）
    - left join:  left outer join
    - right join: right outer join
    
    ```sql
    -- outer join 分为left outer join 和 right outer join
    -- 左表 join 右表
    SELECT * FROM employees e LEFT JOIN salaries s  -- 结果：56行
on e.emp_no = s.emp_no;  -- 左外连接 左表所有行全部出来 然后做等值连接 没有匹配的 悬空

    SELECT * FROM employees e RIGHT JOIN salaries s  -- 结果： 40行
on e.emp_no = s.emp_no;  -- 右外连接 输出顺序不变

    SELECT e.emp_no, s.salary FROM salaries s RIGHT JOIN employees e
ON e.emp_no = s.emp_no;

    ```
  - self JOIN  自连接
  
    self JOIN 层级部门表 层级员工表
    ```bash

    staff 员工表
    id      name        mgr_id
    1       tom         null
    2       jerry       1
    3       ben         1
    4       sam         3
    ```
    
    找：ben的领导的名字？
    ```sql
    select name from staff where id in (select mgr_id from staff where name = 'ben');
    
    -- 自关联
    SELECT mgr.id, mgr.`name` FROM staff mgr JOIN staff s
ON s.mgr_id = mgr.id
WHERE s.`name` = 'ben';

    -- 有领导的员工
    select * from manager where mgr is not NULL
    ```

* ORM
  - ORM 对象关系映射 对象和关系之间的映射 使用面向对象的方式操作数据库
  Object/Relational Mapping  -- 对象关系映射
 
  关系模型和Python对象之前的映射
  table  -->  class    表映射为类
  row    -->  object   行映射为实例
  cloumn -->  property 字段映射为属性
  
  模块化：模块只加载一次 模块顶层代码中的class也只加载一次 类属性(类字段)也只有一次定义 def __init__(self): pass  每一个实例 实例化之后 实例自己的属性各不相同

* Django ORM

WSGI --> ASGI


```bash
pip install django==x.x.x  # 安装django
django-admin startproject name [dir]  # 使用脚手架工具 创建项目相关文件


django-admin startproject first .

django本身不是Server(不是 nginx|httpd) 只是个WSGI App 需要配合Server调用它（dango本质 只是个函数 函数入口在：wsgi.py or asgi.py调用）

python manage.py startapp user  
# 创建一个业务包 项目中的一个应用 一个功能模块
# 创建包之后 一定要在 项目settings.py文件中 注册一下：配置 INSTALLED_APPS = [..., 'user',]   非常重要


# 目录结构
WEBTest  # 项目目录
  -- first  # 主目录 主要是项目配置、入口
    -- __init__.py  # 包文件
    -- asgi.py  # 2.x 不支持asgi 各种gi的入口文件
    -- settings.py  # 全局配置文件 重要
    -- urls.py  # 路径映射
    -- wsgi.py  # 入口
  
  -- user  # 和fisrt平行 功能模块
    -- migrations  # 数据库迁移相关的目录
      -- __init__.py
    -- __init__.py  # 包文件
    -- admin.py  # 后台管理
    -- apps.py  # 跟该应用相关的数据
    -- models.py  # ORM构建模型类
    -- tests.py  # 测试脚本
    -- views.py  # 视图
    
  -- manage.py  # 相当于 django-admin 代理

```

* ORM 查询

* 结果集查询
限制查询集(切片)
分页功能实现 使用限制查询集

查询集对象可以直接使用索引下标的方式（不支持负索引） 相当于SQL语句中的LIMIT和OFFSET子句
注意使用索引返回的新的结果集 依然是惰性求值 不会立刻查询


注: 在使用print函数打印结果集的时候 看到SQL语句有自动添加的LIMIT 21 这是怕打印的太长 使用for循环迭代就没

```python
qs = Employee.objects.all()[10:15]  # limit:5 offset:10

qs = Employee.objects.all()[20:30]  # limit:10 offset:20

```


* 结果集方法
   ```python

    # filter exclude order_by 都返回查询集

    x = emgr.filter(emp_no=10010)  # filter  --> where 关键字传参

    x = emgr.exclude(emp_no=10010)  # exclude --> where not 关键字传参;反选 慎重使用

    x = emgr.order_by('-gender')  # 位置传参

    x = emgr.values()  # 返回查询集 但是内容是字典 values：对象封装->字典封装

    x = emgr.filter(pk=10010).values('emp_no', 'gender')  # pk等于主键, values 指定投影字段

    # 综合 values可以放后面
    x = emgr.exclude(pk=10010).values('emp_no', 'gender').order_by('-pk', 'gender')
    x = emgr.exclude(pk=10010).order_by('-pk', 'gender').values('emp_no', 'gender')

    print(type(x))
    print(x)
    ```
    
 * 返回单个值
     ```python

    x = emgr.filter(pk=10001).get()  # get要求结果集中严格的只能返回1个对象 返回的对象
    x = emgr.get(pk=10001)  # get 后面必须跟唯一键 保证唯一 使用异常捕获来处理 保证不会出错
    x = emgr.filter(pk=10001).first()  # 为空返回None 不抛异常；对象 or None
    x = emgr.filter(pk=10001).last()
    x = emgr.all().last()

    # count 使用主键计算 速度很快 可以使用
    x = emgr.count()    # args=() 参数化查询 防SQL注入
    x = emgr.all().exists()  # 判断查询结果集中是否有数据 如果有返回True
    print(type(x))
    print(x)
    ```
    
 * 字段查询  Field lookup
 
    ```python

    x = emgr.filter(pk__lte=10005)
    x = emgr.exclude(pk__lte=10005)
    x = emgr.filter(last_name__istartswith='p')
    x = emgr.filter(pk__in=[10010, 10018])  # =10010 or =10018
    x = emgr.exclude(pk__in=[10010, 10018])  # =10010 or =10018
    print(type(x))
    print(x)  
    ```
    
    ```python
    
    # and
    x = emgr.filter(pk__gt=10008, pk__lt=10011)
    x = emgr.filter(pk__gt=10008).filter(pk__lt=10011)
    x = emgr.filter(pk__gt=10008) & emgr.filter(pk__lt=10011)
    # Q对象 and &
    x = emgr.filter(Q(pk__gt=10008), Q(pk__lt=10011))
    x = emgr.filter(Q(pk__gt=10008) & Q(pk__lt=10011))

    # or
    x = emgr.filter(pk__in=[10010, 10001])
    x = emgr.filter(pk__lt=10003) | emgr.filter(pk__gt=10018)
    # Q对象 or |
    x = emgr.filter(Q(pk__lt=10003) | Q(pk__gt=10018))

    # Q对象 非  ~ 对Q对象取反
    x = emgr.filter(~Q(pk__gte=10003))

    print(type(x))
    print(x)
    ```
    
 * 聚合 分组
     
    ```python
    from django.db.models import Q, Count, Max, Min, Sum, Avg
    # aggregate 聚合 返回字典 方便使用
    x = emgr.filter(pk__lt=10006).count()
    x = emgr.filter(pk__lt=10006).aggregate(Count('pk'), Sum('pk'))  # 提供聚合函数
    x = emgr.filter(pk__lt=10006).aggregate(count=Count('pk'), sum=Sum('pk'))  # 提供聚合函数 提供别名

    # annotate分组聚合 返回查询集
    x = emgr.filter(pk__lt=10006).values('gender').annotate(count=Count('pk'), sum=Sum('pk'))
    # values 和annotate()配合 指定分组(group by)字段;
    # 放在Query集合最后 指定select 投影字段
    x = emgr.filter(pk__lt=10006).values('gender').annotate(count=Count('pk'), sum=Sum('pk')).values('count').order_by('count')

    print(type(x))
    print(x)
    ```
 

* Django ORM 多表玩法

```python
smgr = Salary.objects
print(Salary.__dict__.keys())
print(Employee.__dict__.keys())  # 'salary_set' -> 'ss'
print('=' * 30)

# raw调用原生SQL
# # salary > 55000
# x = emgr.raw(
#     """
#     SELECT DISTINCT e.emp_no, e.first_name, e.last_name, s.salary FROM employees e JOIN salaries s
# ON e.emp_no = s.emp_no
# WHERE s.salary > 55000
#     """
# )
# print(type(x))
# for s in x:
#     # print(s.__dict__)
#     print(s.name, s.salary)

# distinct 去重 不返回重复记录
# emps = smgr.values('emp_no').distinct()
# print(emps)
# print(emgr.filter(pk__in=emps))  # 子查询实现
# print(emgr.filter(pk__in=[e['emp_no'] for e in emps]))  # IN (x, xx, xxx)



# 查询：10004 工资
# 1. E -> S 从员工表查询
# x = emgr.get(pk=10004)
# x = list(emgr.filter(pk__lte=10004))
# print(type(x))
# print(x)
# print('-' * 30)
# for s in x:
#     print(s.ss.values('salary'))

# 2. S -> E 从工资表查询  别扭，不好用
# 使用list将结果集包起来 会有缓存 只会查询一次
# x = list(smgr.filter(emp_no=10004))
# x = list(smgr.filter(emp_no__lte=10004))
# print(type(x))
# print(x)
# print('-' * 30)
# flag = False
# name = ''
# for s in x:
#     if not flag:
#         first = s.emp_no
#         name = first.name
#         flag = True
#     print(name, s.salary)
```